### Clean up resources

In [ ]:
import boto3

### This deletes all IAM policies beginning with X

In [ ]:
def delete_policies_with_prefixes(policy_prefixes):
    """
    Delete IAM policies that start with specified prefixes
    """
    iam = boto3.client('iam')
    
    try:
        # List all policies
        paginator = iam.get_paginator('list_policies')
        
        # We only want customer managed policies (not AWS managed)
        page_iterator = paginator.paginate(Scope='Local')
        
        for page in page_iterator:
            for policy in page['Policies']:
                policy_name = policy['PolicyName']
                policy_arn = policy['Arn']
                
                # Check if policy name starts with any of the given prefixes
                if any(policy_name.startswith(prefix) for prefix in policy_prefixes):
                    try:
                        # First, delete all versions except the default version
                        versions = iam.list_policy_versions(PolicyArn=policy_arn)['Versions']
                        for version in versions:
                            if not version['IsDefaultVersion']:
                                iam.delete_policy_version(
                                    PolicyArn=policy_arn,
                                    VersionId=version['VersionId']
                                )
                        
                        # Then delete the policy itself
                        iam.delete_policy(PolicyArn=policy_arn)
                        print(f"Successfully deleted policy: {policy_name}")
                        
                    except iam.exceptions.DeleteConflictException:
                        print(f"Could not delete {policy_name} - policy is attached to users/roles")
                    except Exception as e:
                        print(f"Error deleting {policy_name}: {str(e)}")
                        
    except Exception as e:
        print(f"Error: {str(e)}")

In [ ]:
# Usage example
prefixes_to_delete = [
    "KnowledgeBaseQuickCreateAurora"
]

delete_policies_with_prefixes(prefixes_to_delete)

### This deletes all IAM policies beginning with X, that are not attached to any IAM Role

In [ ]:
import boto3
from botocore.exceptions import ClientError

def get_policies_by_prefix(prefix):
    """Get all customer-managed policies that start with the given prefix"""
    iam = boto3.client('iam')
    policies = []
    
    try:
        paginator = iam.get_paginator('list_policies')
        # Scope: Local means customer-managed policies only (not AWS-managed)
        for page in paginator.paginate(Scope='Local'):
            for policy in page['Policies']:
                if policy['PolicyName'].startswith(prefix):
                    print(f"Checking if policy: {policy['PolicyName']} is attached to a Role")                
                    policies.append({
                        'PolicyArn': policy['Arn'],
                        'PolicyName': policy['PolicyName']
                    })
    except ClientError as e:
        print(f"Error listing policies: {e}")
        return []
        
    return policies

def is_policy_attached(policy_arn):
    """Check if policy is attached to any IAM role"""
    iam = boto3.client('iam')
    
    try:
        # Get the number of entities (roles, users, groups) the policy is attached to
        response = iam.get_policy(PolicyArn=policy_arn)
        attachment_count = response['Policy']['AttachmentCount']
        return attachment_count > 0
    except ClientError as e:
        print(f"Error checking policy attachment: {e}")
        return True  # Return True as a safety measure

def delete_unattached_policies(prefix):
    """Delete all unattached policies that start with the given prefix"""
    iam = boto3.client('iam')
    policies = get_policies_by_prefix(prefix)
    
    if not policies:
        print(f"No policies found with prefix '{prefix}'")
        return
    
    for policy in policies:
        policy_arn = policy['PolicyArn']
        policy_name = policy['PolicyName']
        
        if not is_policy_attached(policy_arn):
            try:
                # If Policy has N versions
                versions = iam.list_policy_versions(PolicyArn=policy_arn)['Versions']
                for version in versions:
                    if not version['IsDefaultVersion']:
                        print(f"Deleting version {version['VersionId']} of policy {policy_name}")
                        iam.delete_policy_version(PolicyArn=policy_arn, VersionId=version['VersionId'])
                
                # Delete IAM policy       
                iam.delete_policy(PolicyArn=policy_arn)
                print(f"Successfully deleted unattached policy: {policy_name}")
            
            except ClientError as e:
                print(f"Error deleting policy {policy_name}: {e}")
        else:
            print(f"Policy {policy_name} is attached to one or more entities - skipping deletion")

In [ ]:
# Usage
if __name__ == "__main__":
    prefix = "bedrock"
    print(f"Searching for unattached policies with prefix '{prefix}'...")
    delete_unattached_policies(prefix)
